## Module 5: Pre-processing Techniques

In this module we’ll explore common audio “clean-up” and conditioning methods used before further analysis or synthesis.

### Key Concepts
- **Denoising**  
  - Spectral subtraction  
  - Wiener filtering  
- **Normalization & Dynamic Range Compression**  
  - Peak vs. RMS normalization  
  - Compressor parameters: threshold, ratio, attack/release  
- **Filtering**  
  - FIR vs. IIR filters  
  - Window design (Hamming, Blackman, etc.)  

---

### 📓 Notebook Demos

1. **Spectral Subtraction Denoising**  
   - Manually set your noise segment and subtraction gain in the code  
   - Observe the cleaned waveform and spectrum  

2. **Wiener-Filter Denoising**  
   - Use the same noise estimate to design a Wiener filter  
   - Compare audio and spectrograms to see the differences  

3. **Dynamic Range Compression**  
   - Manually set compressor threshold, ratio, attack & release in code  
   - Play back and plot gain-reduction curves  

4. **FIR vs. IIR Filtering & Window Design**  
   - In a final demo cell, specify cutoff, filter type, and window (in code)  
   - Plot impulse responses and magnitude responses of:  
     - An FIR low-pass (using Hamming, Blackman, etc.)  
     - An IIR Butterworth low-pass  
   - Apply each to a test clip and play the results  

---

### 🛠 Exercise: Mains-Hum Removal Filter
- **Task:** Implement a band-stop (notch) filter at 50 Hz (or 60 Hz) to remove mains hum from a voice recording.  
- **Steps:**  
  1. Design a narrow IIR notch filter centered at the line frequency.  
  2. Apply it to a WAV file containing speech + hum.  
  3. Compare spectra and audio before/after filtering.  
- **Deliverable:**  
  - Plot the magnitude response of your notch filter  
  - Show spectrograms pre-/post-filter  
  - Include audio players for before/after listening  


### Key Concepts: Denoising

Before diving into the hands-on demos, it’s important to understand the two primary denoising strategies we’ll be exploring:

- **Spectral Subtraction**  
  1. **Noise Estimate:**  You first select a short “noise-only” segment of your recording (silence plus background noise).  
  2. **Spectrum Subtraction:**  Compute the magnitude spectrum of that noise segment, then subtract it (often scaled by a user-controlled gain) from the magnitude spectrum of the full signal.  
  3. **Reconstruction:**  Combine the cleaned magnitude with the original phase and invert back to the time domain.  
  <br>  
  **Key Points:**  
  - Easy to implement and tune via a single “subtraction gain.”  
  - Can produce musical noise (artifacts) if over-subtracted.

- **Wiener Filtering**  
  1. **Signal & Noise PSD:**  Estimate the power spectral density (PSD) of both the noise and the noisy signal.  
  2. **Filter Design:**  Compute a frequency-dependent gain \(H(f)\) that minimizes mean-square error between the clean and estimated signals.  
     \[
       H(f) \;=\; \frac{S_{xx}(f)}{S_{xx}(f) + S_{nn}(f)}
     \]
     where \(S_{xx}\) is the clean-signal PSD and \(S_{nn}\) is the noise PSD.  
  3. **Apply & Reconstruct:**  Multiply the noisy signal’s spectrum by \(H(f)\), then invert to time domain.  
  <br>  
  **Key Points:**  
  - Statistically optimal under Gaussian noise assumptions.  
  - Tends to introduce fewer “musical” artifacts than spectral subtraction, but may sound “muffled” if PSD estimates are poor.

In the upcoming demos you’ll manually set the noise estimate and filter parameters in the code, then compare the results of spectral subtraction vs. Wiener filtering on a noisy clip.  


## Demo 1: Spectral Subtraction Denoising

In this demo you’ll remove a stationary noise floor from an audio clip using **spectral subtraction**.

### What the code does
1. **Loads** your noisy audio file.  
2. **Computes** an STFT to extract magnitude & phase.  
3. **Estimates** the average noise spectrum from a user-specified noise-only segment.  
4. **Subtracts** (with gain) that noise spectrum from every frame’s magnitude.  
5. **Reconstructs** the denoised waveform by combining the cleaned magnitudes with the original phase.  
6. **Plays back** both the noisy input and the denoised output.  
7. **Plots** time-domain waveforms (before vs. after) and average magnitude spectra.  

---

### How to use
1. **Edit the USER SETTINGS** at the top of the code cell:  
   - `FILENAME` – your noisy clip in `sounds/` (supports WAV/MP3).  
   - `NOISE_START`, `NOISE_END` – start/end times (s) of a pure noise segment.  
   - `SUB_GAIN` – subtraction gain (e.g. 0.5…2.0).  
   - `N_FFT` / `HOP_LENGTH` – STFT window & hop sizes (powers of two recommended).  
2. **Run** the cell.  

---

### What to observe

#### Audio
- Does the **denoised** version sound cleaner?  
- Are any artifacts (“musical noise”) introduced when you increase `SUB_GAIN`?

#### Waveforms
- The denoised signal should show **reduced background noise** in quiet regions.

#### Spectra
- The average magnitude spectrum after subtraction should have a **lowered noise floor** across frequencies.


In [ ]:
# ── USER SETTINGS ────────────────────────────────────────────────────────────────
FILENAME      = 'noisy_clip.mp3'   # ← place your noisy audio clip in `sounds/`
NOISE_START   = 0.0                # ← start time (s) of a noise-only segment
NOISE_END     = 0.5                # ← end time (s) of the noise-only segment
SUB_GAIN      = 1.0                # ← subtraction gain (e.g. 0.5…2.0)
N_FFT         = 1024               # ← FFT window size
HOP_LENGTH    = N_FFT // 4         # ← hop length for STFT
# ────────────────────────────────────────────────────────────────────────────────

import numpy as np
import matplotlib.pyplot as plt
import librosa
from IPython.display import Audio, display
from pathlib import Path

# ── CONFIG (don’t edit below here) ───────────────────────────────────────────────
SOUNDS_DIR = Path('sounds')
audio_path = SOUNDS_DIR / FILENAME

# 1) Load audio
y, sr = librosa.load(str(audio_path), sr=None)

# 2) Compute STFT
D      = librosa.stft(y, n_fft=N_FFT, hop_length=HOP_LENGTH)
mag    = np.abs(D)
phase  = np.angle(D)

# 3) Estimate noise spectrum from noise-only segment
start_frame = int(NOISE_START * sr / HOP_LENGTH)
end_frame   = int(NOISE_END   * sr / HOP_LENGTH)
noise_mag   = mag[:, start_frame:end_frame].mean(axis=1, keepdims=True)

# 4) Spectral subtraction
mag_clean = mag - SUB_GAIN * noise_mag
mag_clean = np.clip(mag_clean, a_min=0.0, a_max=None)

# 5) Reconstruct complex spectrogram and invert to time-domain
D_clean = mag_clean * np.exp(1j * phase)
y_clean = librosa.istft(D_clean, hop_length=HOP_LENGTH)

# 6) Display audio players
print("▶️ Original (Noisy) Audio")
display(Audio(data=y,       rate=sr, autoplay=False))
print("▶️ Denoised Audio")
display(Audio(data=y_clean, rate=sr, autoplay=False))

# 7) Plot time-domain waveforms
times = np.linspace(0, len(y)/sr, len(y))
plt.figure(figsize=(12, 4))
plt.plot(times, y,       label='Noisy', alpha=0.7)
plt.plot(times, y_clean, label='Denoised', alpha=0.7)
plt.title('Waveform: Before vs. After Spectral Subtraction')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 8) Plot magnitude spectra (averaged over time)
mag_orig_mean = mag.mean(axis=1)
mag_clean_mean = mag_clean.mean(axis=1)
freqs = np.linspace(0, sr/2, len(mag_orig_mean))

plt.figure(figsize=(12, 4))
plt.plot(freqs, 20*np.log10(mag_orig_mean+1e-8),   label='Noisy', alpha=0.7)
plt.plot(freqs, 20*np.log10(mag_clean_mean+1e-8), label='Denoised', alpha=0.7)
plt.title('Average Magnitude Spectrum: Before vs. After')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude (dB)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


## Demo 2: Wiener‐Filter Denoising

In this demo you’ll apply a classical Wiener filter to reduce stationary noise in an audio clip.

**What the code does:**  
1. **Loads** your noisy audio file.  
2. **Computes** its STFT (magnitude & phase).  
3. **Estimates** the average noise spectrum from your specified noise-only segment.  
4. **Designs** a Wiener gain mask:  
   \[
     G(k,n) = \frac{|S(k,n)|^2}{|S(k,n)|^2 + |N(k)|^2}
   \]
5. **Applies** the gain mask to the complex spectrogram and reconstructs the time-domain signal.  
6. **Plays back** both the original noisy clip and the Wiener-denoised result.  
7. **Plots** side-by-side log-frequency spectrograms before vs. after filtering.

---

### How to use

1. **Edit the USER SETTINGS** at the top of the code cell:  
   - `FILENAME` – your noisy clip in the `sounds/` folder (WAV or MP3)  
   - `NOISE_START`, `NOISE_END` – start/end times (seconds) of a purely noise segment  
   - `N_FFT` – STFT window size (power of two)  
   - `HOP_LENGTH` – hop size between frames (usually `N_FFT/4`)  

2. **Run the cell** to perform the Wiener filtering.

---

### What to observe

- **Audio:**  
  - ▶️ Play the **Original (Noisy)** and **Wiener-Denoised** versions.  
  - Listen for reduced hiss or background noise, and note any filtering artifacts.

- **Spectrograms:**  
  - The **Noisy** spectrogram shows broadband noise across frequencies.  
  - The **Wiener-Denoised** spectrogram should exhibit attenuated noise floor and cleaner harmonic structures.

- **Trade-off:**  
  - Strong noise reduction may introduce “musical noise” or distort transients.  
  - Experiment with different noise segments and STFT settings to balance denoising vs. artifacting.  


In [ ]:
# ── USER SETTINGS ────────────────────────────────────────────────────────────────
FILENAME     = 'noisy_clip.mp3'   # ← place your noisy clip in `sounds/` (WAV or MP3)
NOISE_START  = 0.0                # ← start time (s) of a noise-only segment
NOISE_END    = 0.5                # ← end time (s) of the noise-only segment
N_FFT        = 1024               # ← STFT window size (power of two)
HOP_LENGTH   = N_FFT // 4         # ← hop length for STFT
# ────────────────────────────────────────────────────────────────────────────────

import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
from IPython.display import Audio, display
from pathlib import Path

# ── CONFIG (don’t edit below here) ───────────────────────────────────────────────
SOUNDS_DIR = Path('sounds')
audio_path = SOUNDS_DIR / FILENAME

# 1) Load audio
y, sr = librosa.load(str(audio_path), sr=None)

# 2) Compute STFT
D      = librosa.stft(y, n_fft=N_FFT, hop_length=HOP_LENGTH)
mag    = np.abs(D)
phase  = np.angle(D)

# 3) Estimate noise spectrum
start_frame = int(NOISE_START * sr / HOP_LENGTH)
end_frame   = int(NOISE_END   * sr / HOP_LENGTH)
noise_mag   = mag[:, start_frame:end_frame].mean(axis=1, keepdims=True)

# 4) Design and apply Wiener filter
#    Wiener gain = |S|^2 / (|S|^2 + |N|^2)
gain = (mag**2) / (mag**2 + noise_mag**2 + 1e-8)
D_wiener = gain * D
y_wiener = librosa.istft(D_wiener, hop_length=HOP_LENGTH)

# 5) Audio playback
print("▶️ Original (Noisy) Audio")
display(Audio(data=y,         rate=sr, autoplay=False))
print("▶️ Wiener-Denoised Audio")
display(Audio(data=y_wiener,  rate=sr, autoplay=False))

# 6) Plot spectrograms
fig, ax = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

# Original spectrogram
S_db = librosa.amplitude_to_db(mag, ref=np.max)
librosa.display.specshow(S_db, sr=sr, hop_length=HOP_LENGTH, 
                         x_axis='time', y_axis='log', ax=ax[0])
ax[0].set_title('Noisy Spectrogram')
ax[0].invert_yaxis()

# Wiener-denoised spectrogram
mag_wiener = np.abs(D_wiener)
S_w_db = librosa.amplitude_to_db(mag_wiener, ref=np.max)
librosa.display.specshow(S_w_db, sr=sr, hop_length=HOP_LENGTH, 
                         x_axis='time', y_axis='log', ax=ax[1])
ax[1].set_title('Wiener-Denoised Spectrogram')
ax[1].invert_yaxis()

plt.suptitle('Spectrogram: Before vs. After Wiener Filtering', fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


### Key Concepts: Normalization & Dynamic Range Compression

- **Peak vs. RMS Normalization**  
  - **Peak normalization** rescales the entire signal so its maximum absolute sample reaches a target level (e.g. –1 dBFS).  
  - **RMS normalization** adjusts the overall loudness by matching the root-mean-square energy to a target (e.g. –18 LUFS), yielding a more perceptually consistent level.

- **Dynamic Range Compression**  
  A compressor reduces the level of loud passages above a set threshold, “squashing” the dynamic range and bringing quieter and louder parts closer together.  

  - **Threshold**: the level (in dB) above which gain reduction begins.  
  - **Ratio**: how much the signal above threshold is turned down (e.g. 4:1 means 4 dB in → 1 dB out).  
  - **Attack time**: how quickly the compressor kicks in after the signal exceeds the threshold (milliseconds).  
  - **Release time**: how quickly it stops compressing once the signal falls below the threshold (milliseconds).  

Understanding and tuning these parameters allows you to control loudness, punch, and perceived “warmth” or “presence” in your audio.  


## Demo 3: Dynamic Range Compression

In this demo you'll apply a simple compressor to a percussive audio clip and visualize how it tames peaks and reduces dynamic range.

**What the code does:**
1. **Loads** your audio file from `sounds/`.
2. **Normalizes** it so the peak level is 0 dBFS.
3. **Computes** a sample-by-sample envelope follower using your attack & release times.
4. **Applies** compression above your threshold with the specified ratio.
5. **Smooths** the gain changes, adds any makeup gain, and **reconstructs** the compressed signal.
6. **Plays back** both the original and compressed audio.
7. **Plots**  
   - The input vs. output level envelopes over time, with the shaded area showing gain reduction  
   - The static gain-reduction curve (input level → reduction in dB)

**How to use:**
- Edit the **USER SETTINGS** at the top of the code cell:
  - `FILENAME` &mdash; your clip in `sounds/`  
  - `THRESHOLD_DB` &mdash; compressor threshold (dBFS), e.g. -60 … 0  
  - `RATIO` &mdash; how hard to compress above threshold (> 1.0)  
  - `ATTACK_MS`, `RELEASE_MS` &mdash; envelope times in milliseconds (> 0)  
  - `OUTPUT_GAIN_DB` &mdash; any makeup gain after compression (dB)  
- Run the cell.

**What to observe:**
- **Audio A/B** &mdash; does the compressed version sound more even and controlled?  
- **Envelope plot** &mdash; see how peaks above the threshold are “pulled down” by the compressor (shaded area).  
- **Gain curve** &mdash; confirms the static behavior: input levels above `THRESHOLD_DB` are reduced by `(1 – 1/ratio)` of their excess.  


In [ ]:
# ── USER SETTINGS ────────────────────────────────────────────────────────────────
FILENAME        = 'drum_hit3.wav'   # ← place your audio file in `sounds/`
THRESHOLD_DB    = -20.0             # ← compressor threshold in dBFS (e.g. -60 … 0)
RATIO           = 4.0               # ← compression ratio (>1.0)
ATTACK_MS       = 10.0              # ← attack time in milliseconds (>0)
RELEASE_MS      = 100.0             # ← release time in milliseconds (>0)
OUTPUT_GAIN_DB  = 0.0               # ← makeup gain after compression in dB (e.g. -6 … +6)
# ────────────────────────────────────────────────────────────────────────────────

import numpy as np
import librosa
import matplotlib.pyplot as plt
from IPython.display import Audio, display
from pathlib import Path

# ── CONFIG (don’t edit below here) ───────────────────────────────────────────────
SOUNDS_DIR = Path('sounds')
y, sr = librosa.load(str(SOUNDS_DIR / FILENAME), sr=None)
# normalize to peak = 1
y = y / np.max(np.abs(y) + 1e-16)

# envelope follower coefficients
attack_tc  = np.exp(-1.0 / (sr * (ATTACK_MS  / 1000.0)))
release_tc = np.exp(-1.0 / (sr * (RELEASE_MS / 1000.0)))

# prepare arrays
env      = np.zeros_like(y)
gain_db  = np.zeros_like(y)
prev_env = 0.0

# compute envelope and static gain
for n, sample in enumerate(y):
    x = abs(sample)
    if x > prev_env:
        prev_env = attack_tc  * prev_env + (1-attack_tc)  * x
    else:
        prev_env = release_tc * prev_env + (1-release_tc) * x
    env[n] = prev_env
    level_db = 20 * np.log10(prev_env + 1e-8)
    if level_db > THRESHOLD_DB:
        # apply compression above threshold
        comp_db = THRESHOLD_DB + (level_db - THRESHOLD_DB)/RATIO
        gain_db[n] = comp_db - level_db
    else:
        gain_db[n] = 0.0

# smooth gain with release (optional but recommended)
smoothed_gain_db = np.copy(gain_db)
prev_g = 0.0
for n, g in enumerate(gain_db):
    if g < prev_g:
        prev_g = attack_tc  * prev_g + (1-attack_tc)  * g
    else:
        prev_g = release_tc * prev_g + (1-release_tc) * g
    smoothed_gain_db[n] = prev_g

# apply makeup gain
total_gain_db = smoothed_gain_db + OUTPUT_GAIN_DB
total_gain_lin = 10**(total_gain_db / 20)

y_comp = y * total_gain_lin

# ── PLAYBACK ───────────────────────────────────────────────────────────────────
print("▶️ Original Audio")
display(Audio(data=y,       rate=sr, autoplay=False))
print("▶️ Compressed Audio")
display(Audio(data=y_comp,  rate=sr, autoplay=False))

# ── PLOTS ───────────────────────────────────────────────────────────────────────
times = np.arange(len(y)) / sr

# 1) Envelope and gain reduction
plt.figure(figsize=(10, 4))
plt.plot(times, 20*np.log10(env + 1e-8),      label='Input Level (dBFS)')
plt.plot(times, 20*np.log10(env + 1e-8) + smoothed_gain_db, 
         label='Output Level (dBFS)')
plt.fill_between(times, 
                 20*np.log10(env + 1e-8), 
                 20*np.log10(env + 1e-8) + smoothed_gain_db,
                 color='C1', alpha=0.3,
                 label='Gain Reduction')
plt.title('Compressor Envelope & Gain Reduction')
plt.xlabel('Time (s)')
plt.ylabel('Level (dB)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 2) Gain reduction curve
plt.figure(figsize=(6,4))
input_levels = np.linspace(-80, 0, 100)
gain_curve = np.where(
    input_levels > THRESHOLD_DB,
    (THRESHOLD_DB + (input_levels - THRESHOLD_DB)/RATIO) - input_levels,
    0.0
)
plt.plot(input_levels, gain_curve, linewidth=2)
plt.title('Static Gain Reduction Curve')
plt.xlabel('Input Level (dBFS)')
plt.ylabel('Gain Reduction (dB)')
plt.grid(True)
plt.tight_layout()
plt.show()


### Key Concept: Filtering

- **FIR vs. IIR Filters**  
  - **FIR (Finite Impulse Response):**  
    - Non-recursive, inherently stable, linear‐phase (if symmetric taps).  
    - Design by placing a window on an ideal impulse response.  
  - **IIR (Infinite Impulse Response):**  
    - Recursive, can achieve sharp cutoffs with fewer coefficients.  
    - Generally nonlinear phase, may require care for stability.

- **Window Design**  
  - **Rectangular (boxcar):**  
    - Simple, narrow main-lobe but large side-lobes (poor stop-band attenuation).  
  - **Hamming / Hann:**  
    - Wider main-lobe, much lower side-lobes (better suppression of distant frequencies).  
  - **Blackman / Blackman–Harris:**  
    - Even wider main-lobe, extremely low side-lobes (excellent stop-band performance).

Filtering trade-offs are often between transition-band width (main-lobe) vs. stop-band attenuation (side-lobes) and phase behavior.  


## Demo 4: FIR vs. IIR Low-Pass Filtering

In this demo you’ll design and compare two simple low-pass filters—a windowed FIR filter and a Butterworth IIR filter—and apply them to an audio clip.

### What the code does:
1. **Load** your chosen audio file (normalized to peak = 1).  
2. **Design**:  
   - An **FIR** filter of order `FIR_ORDER` with cutoff `CUTOFF_FREQ` Hz using the specified `WINDOW_TYPE`.  
   - An **IIR** Butterworth filter of order `IIR_ORDER` with the same cutoff.  
3. **Plot** their impulse responses side by side, so you can see the FIR taps vs. the IIR’s recursive response.  
4. **Plot** their magnitude responses (frequency-domain) on the same axes to compare pass-band flatness and stop-band attenuation.  
5. **Apply** both filters (zero-phase via `filtfilt`) to your clip and display audio players for:  
   - Original  
   - FIR-filtered  
   - IIR-filtered  
6. **Plot** the first 0.1 s of each waveform overlaid to illustrate how each filter shapes the time-domain signal.

---

### How to use:
- Edit the **USER SETTINGS** at the top of the code cell:  
  - `FILENAME`: name of your test audio file in `sounds/`.  
  - `CUTOFF_FREQ`: cutoff frequency in Hz (must satisfy `0 < CUTOFF_FREQ < sr/2`).  
  - `FIR_ORDER`: order of the FIR filter (even integer ≥ 2).  
  - `WINDOW_TYPE`: window for the FIR design (`'boxcar'`, `'hann'`, `'hamming'`, `'blackman'`).  
  - `IIR_ORDER`: order of the IIR Butterworth filter (integer ≥ 1).  
- Run the cell to:  
  1. See the impulse-response and frequency-response plots.  
  2. Listen to the original, FIR-filtered, and IIR-filtered audio.  
  3. Inspect the overlaid waveform for the first 0.1 s.


In [ ]:
# ── USER SETTINGS ────────────────────────────────────────────────────────────────
FILENAME      = 'drum_hit3.wav'   # ← place your test audio file in `sounds/`
CUTOFF_FREQ   = 1000.0            # ← cutoff frequency in Hz (0 < CUTOFF_FREQ < sr/2)
FIR_ORDER     = 64                # ← order of FIR filter (even integer ≥ 2)
WINDOW_TYPE   = 'hamming'         # ← window for FIR ('boxcar','hann','hamming','blackman')
IIR_ORDER     = 4                 # ← order of IIR Butterworth filter (integer ≥ 1)
# ────────────────────────────────────────────────────────────────────────────────

import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import firwin, butter, freqz, filtfilt
from IPython.display import Audio, display
import librosa
from pathlib import Path

# ── CONFIG (don’t edit below here) ───────────────────────────────────────────────
SOUNDS_DIR = Path('sounds')
audio_path = SOUNDS_DIR / FILENAME

# 1) Load audio
y, sr = librosa.load(str(audio_path), sr=None)
y = y / np.max(np.abs(y) + 1e-16)  # normalize peak = 1

# 2) Design FIR low-pass filter
nyq = sr / 2
fir_coeff = firwin(
    numtaps=FIR_ORDER + 1,
    cutoff=CUTOFF_FREQ / nyq,
    window=WINDOW_TYPE
)

# 3) Design IIR Butterworth low-pass filter
b_iir, a_iir = butter(
    N=IIR_ORDER,
    Wn=CUTOFF_FREQ / nyq,
    btype='lowpass'
)

# 4) Plot impulse responses
plt.figure(figsize=(10,3))
plt.stem(np.arange(len(fir_coeff)), fir_coeff, linefmt='C0-', markerfmt='C0o', basefmt=" ", label='FIR taps')
imp_iir = np.zeros(len(fir_coeff))
imp_iir[0] = 1.0
h_iir = filtfilt(b_iir, a_iir, imp_iir)
plt.plot(h_iir, 'C1-', linewidth=2, label='IIR impulse response')
plt.title('Impulse Responses')
plt.xlabel('Samples')
plt.ylabel('Amplitude')
plt.legend()
plt.grid(True)
plt.show()

# 5) Plot frequency responses
w_fir, h_fir = freqz(fir_coeff, worN=8000)
w_iir, h_iir = freqz(b_iir, a_iir, worN=8000)

plt.figure(figsize=(10,3))
plt.plot(w_fir * sr / (2*np.pi), 20*np.log10(np.abs(h_fir) + 1e-8), label='FIR')
plt.plot(w_iir * sr / (2*np.pi), 20*np.log10(np.abs(h_iir) + 1e-8), label='IIR')
plt.title('Magnitude Responses')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude (dB)')
plt.legend()
plt.grid(True)
plt.ylim(-80, 5)
plt.show()

# 6) Apply filters (zero-phase)
y_fir = filtfilt(fir_coeff, [1.0], y)
y_iir = filtfilt(b_iir, a_iir, y)

# 7) Playback
print("▶️ Original Audio")
display(Audio(data=y,      rate=sr, autoplay=False))
print("▶️ FIR-Filtered Audio")
display(Audio(data=y_fir,  rate=sr, autoplay=False))
print("▶️ IIR-Filtered Audio")
display(Audio(data=y_iir,  rate=sr, autoplay=False))

# 8) Plot filtered waveforms (first 0.1s)
t = np.arange(len(y)) / sr
n_plot = int(0.1 * sr)
plt.figure(figsize=(10,3))
plt.plot(t[:n_plot], y[:n_plot],   label='Original', alpha=0.7)
plt.plot(t[:n_plot], y_fir[:n_plot],label='FIR',      alpha=0.7)
plt.plot(t[:n_plot], y_iir[:n_plot],label='IIR',      alpha=0.7)
plt.title('Waveform Comparison (first 0.1s)')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


### 🛠 Exercise: Mains-Hum Removal Filter

**Task:**  
Implement a band-stop (notch) filter to remove mains hum (50 Hz or 60 Hz) from a voice recording.

**Steps:**
1. **Design** a narrow IIR notch filter centered at the line frequency (50 Hz or 60 Hz).  
2. **Load** a WAV file containing speech with mains hum.  
3. **Apply** your notch filter to the noisy signal.  
4. **Compare** before/after by:  
   - Plotting the **magnitude response** of your notch filter.  
   - Displaying **spectrograms** of the noisy vs. filtered audio.  
   - Playing back both versions with the built-in audio players.

**Deliverables:**
- A plot of your notch filter’s frequency response, showing deep attenuation at the target line frequency.  
- Two spectrograms (pre- and post-filter) highlighting the removed hum component.  
- Audio players to listen to the original vs. filtered signal so you can confirm the hum has been attenuated without severely affecting speech quality.  
